# 교차 검증과 그리드 서치

## 검증 세트


In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine-date')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy() # 2차원 입력 데이터
target = wine['class'].to_numpy() # 1차원 정답 데이터

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42) # 1차: 훈련 세트와 테스트 세트로 나눔 8:2

In [6]:
print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42) # 2차: 5197 데이터를 다시 8:2로 나눔

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


## 교차 검증
* 데이터를 여러 번 나누어 검증하는 방법
* 번갈아가며 진행됨
* 검증 단위: 폴드 (기본 폴드 단위 5-fold)


In [7]:
# 의사결정 트리 평가 (훈련 + 검증)
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


* 훈련 0.99
* 검증 0.86

교차검증으로 둘 차 줄이기?

In [8]:
# 교차 검증: cross_validate (기본 fold: 5)
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.04059696, 0.0319376 , 0.03711772, 0.05507445, 0.05406189]), 'score_time': array([0.01227403, 0.00180268, 0.00175667, 0.0017941 , 0.00183582]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


{'fit_time': array([0.04059696, 0.0319376 , 0.03711772, 0.05507445, 0.05406189]),

'score_time': array([0.01227403, 0.00180268, 0.00175667, 0.0017941 , 0.00183582]),

'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [9]:
# 각 훈련 셋을 검증한 모델 평가 점수를 평균 냄
import numpy as np
print(np.mean(scores['test_score']))


0.855300214703487


In [10]:
# 교차 검증 시 훈련 세트를 섞어서 분할해 주는 분할기 기능
# 1. stratifield (Fold: 분류, KFold: 회귀)
# 2. 임의의 숫자로
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


➡ 교차 검증을 통해 평균 성능 점수 계산

In [11]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


➡ 위 코드와 데이터 나누는 부분에서 차이 有
  * `n_splits=10`: 10개의 폴드로 나누어 교차 검증을 수행
  * 위 코드는 기본 값 5로 나누어져 있지만 아래 코드는 10개로 설정되어 있음
  * 또 무작위성을 더한 부분이 다름

## 하이퍼파라미터 튜닝
* 파라미터를 이용해 모델 성능 평가

### 그리드 서치
* 교차 검증 + 하이퍼파라미터 튜닝 = 그리드 서치
* 파라미터 간 모든 가능한 조합을 시도함

In [15]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.001, 0.0001],
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)} # 불순도 값을 바꿔서 해보고자
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [16]:
print(gs.best_params_)

{'max_depth': 19, 'min_impurity_decrease': 0.0001, 'min_samples_split': 2}


모델이 가장 높은 성능을 나타낸 하이퍼파라미터 조합을 보여줌

In [17]:
print(gs.cv_results_['mean_test_score'])

[0.85780355 0.85799604 0.85799604 0.85761105 0.85761105 0.85761105
 0.85818853 0.85876601 0.85876601 0.85876601 0.85876601 0.85876601
 0.85876601 0.85857352 0.85857352 0.85857352 0.85857352 0.85857352
 0.85857352 0.85857352 0.85780355 0.85799604 0.85799604 0.85761105
 0.85761105 0.85761105 0.85818853 0.85876601 0.85876601 0.85876601
 0.85645721 0.8566497  0.8574193  0.85761124 0.85915063 0.85915081
 0.85992023 0.86049789 0.86030558 0.86049808 0.85992097 0.85992097
 0.85992097 0.85972847 0.86011346 0.86011346 0.86011346 0.86011346
 0.86049808 0.86069057 0.85645721 0.8566497  0.8574193  0.85761124
 0.85915063 0.85915081 0.85992023 0.86049789 0.86030558 0.86049808
 0.85607093 0.85414637 0.85683923 0.85741597 0.86049511 0.85953376
 0.8604953  0.86088065 0.86068835 0.86222903 0.86049567 0.86049567
 0.86030318 0.86011068 0.86049567 0.85991875 0.85991875 0.85991875
 0.86030336 0.86222903 0.85607093 0.85414637 0.85683923 0.85741597
 0.86049511 0.85953376 0.8604953  0.86088065 0.86068835 0.8622

### 랜덤 서치

In [18]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
params = {'min_impurity_decrease':uniform(0.0001, 0.001),
          'max_depth': randint(5, 20, 1),
          'min_samples_split': randint(2, 100, 10)} # 불순도 값을 바꿔서 해보고자
          # leaf는 빼도 ㄱㅊ
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, n_iter=100, random_state=42)
gs.fit(train_input, train_target)

print(gs.best_params_)

print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(train_input, train_target))

{'max_depth': 10, 'min_impurity_decrease': 0.00044507124802668297, 'min_samples_split': 12}
0.8680006663211668
0.8895516644217818


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
